In [1]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa
!pip install ipython-sql

     |████████████████████████████████| 6.0 MB 16.0 MB/s eta 0:00:01
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp38-cp38-linux_x86_64.whl size=1209497 sha256=88ccf8d1434013e4e9670c249f8fbf13e7e8e5b627f004c2b78eb7f78b61cbc0
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/cb/43/46/fa638f2422554332b7865d600275b24568bf60e76104a94bb4
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.22
    Uninstalling SQLAlchemy-1.4.22:
      Successfully uninstalled SQLAlchemy-1.4.22
     |████████████████████████████████| 42 kB 2.3 MB/s  eta 0:00:01
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13700 sha256=f7a7943b24a4e59509b0b436eca4e18190478856763fac17b62f26913af1dfde
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/48/6d/77/9517cb933af254f51a446f1a5ec9c2be3e45f17384940bce68
Successfully built prettytable


In [2]:
%load_ext sql

In [3]:
%sql ibm_db_sa://yky11434:VuIophu6J3aNC7B5@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb?security=SSL

## Task 1

Display the names of the unique launch sites in the space mission

In [4]:
%%sql 
SELECT COUNT(*) FROM SPACEXTBL;

 * ibm_db_sa://yky11434:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


1
101


In [7]:
%%sql 
SELECT DISTINCT LAUNCH_SITE 
FROM SPACEXTBL;

 * ibm_db_sa://yky11434:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


launch_site
CCAFS LC-40
CCAFS SLC-40
KSC LC-39A
VAFB SLC-4E


## Task 2

Display 5 records where launch sites begin with the string 'CCA'

In [8]:
%%sql
SELECT LAUNCH_SITE 
FROM SPACEXTBL 
WHERE LAUNCH_SITE LIKE 'CCA%'
LIMIT 5;

 * ibm_db_sa://yky11434:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


launch_site
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40


## Task 3

Display the total payload mass carried by boosters launched by NASA (CRS)

In [24]:
%%sql 
SELECT SUM(PAYLOAD_MASS__KG_) AS TOTAL_PAYLOAD_MASS
FROM SPACEXTBL
WHERE CUSTOMER LIKE 'NASA (CRS)'
GROUP BY CUSTOMER;

 * ibm_db_sa://yky11434:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


total_payload_mass
45596


## Task 4

Display average payload mass carried by booster version F9 v1.1

In [26]:
%%sql
SELECT AVG(PAYLOAD_MASS__KG_) AS AVERAGE_PAYLOAD_MASS
FROM SPACEXTBL
WHERE BOOSTER_VERSION LIKE 'F9 v1.1'
GROUP BY BOOSTER_VERSION;

 * ibm_db_sa://yky11434:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


average_payload_mass
2928


## Task 5

List the date when the first successful landing outcome in ground pad was acheived

In [28]:
%%sql
SELECT LANDING__OUTCOME, MIN(DATE) AS FIRST_SUCCESSFUL_LANDING
FROM SPACEXTBL
WHERE LANDING__OUTCOME LIKE 'Success (ground pad)'
GROUP BY LANDING__OUTCOME
ORDER BY FIRST_SUCCESSFUL_LANDING
fetch first 1 rows only;

 * ibm_db_sa://yky11434:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


landing__outcome,first_successful_landing
Success (ground pad),2015-12-22


## Task 6

List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [46]:
%%sql
SELECT BOOSTER_VERSION,LANDING__OUTCOME,PAYLOAD_MASS__KG_
FROM SPACEXTBL
WHERE (PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000) AND LANDING__OUTCOME LIKE 'Success (drone ship)';

 * ibm_db_sa://yky11434:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


booster_version,landing__outcome,payload_mass__kg_
F9 FT B1022,Success (drone ship),4696
F9 FT B1026,Success (drone ship),4600
F9 FT B1021.2,Success (drone ship),5300
F9 FT B1031.2,Success (drone ship),5200


## Task 7

List the total number of successful and failure mission outcomes

In [52]:
%%sql
SELECT MISSION_OUTCOME, COUNT(*) AS TOTAL_SUCCESS_FAILURE_MISSION_OUTCOMES
FROM SPACEXTBL
GROUP BY MISSION_OUTCOME;

 * ibm_db_sa://yky11434:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


mission_outcome,total_success_failure_mission_outcomes
Failure (in flight),1
Success,99
Success (payload status unclear),1


## Task 8

List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [57]:
%%sql
SELECT BOOSTER_VERSION 
FROM SPACEXTBL 
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL);

 * ibm_db_sa://yky11434:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


booster_version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


## Task 9

List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015

In [59]:
%%sql
SELECT BOOSTER_VERSION,LAUNCH_SITE,LANDING__OUTCOME,DATE
FROM SPACEXTBL
WHERE DATE LIKE '2015%' AND LANDING__OUTCOME LIKE 'Failure (drone ship)';

 * ibm_db_sa://yky11434:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


booster_version,launch_site,landing__outcome,DATE
F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship),2015-01-10
F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship),2015-04-14


## Task 10

Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order

In [64]:
%%sql
SELECT LANDING__OUTCOME, COUNT(LANDING__OUTCOME) AS LANDING_OUTCOME_COUNT
FROM SPACEXTBL 
WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20' 
GROUP BY LANDING__OUTCOME
ORDER BY COUNT(LANDING__OUTCOME) DESC;

 * ibm_db_sa://yky11434:***@fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32731/bludb
Done.


landing__outcome,landing_outcome_count
No attempt,10
Failure (drone ship),5
Success (drone ship),5
Controlled (ocean),3
Success (ground pad),3
Failure (parachute),2
Uncontrolled (ocean),2
Precluded (drone ship),1
